# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse          Shipments  \
0  Jun  1 2022 10:32AM  238523        10         8087384-PR   
1  Jun  1 2022 10:32AM  238523        10            8088547   
2  Jun  1 2022 10:32AM  238523        10            8088548   
3  Jun  1 2022 10:32AM  238523        10            8088549   
4  Jun  1 2022 10:32AM  238523        10            8091144   
5  Jun  1 2022 10:30AM  238522        18  CLRSPC-20975283.1   
6  Jun  1 2022 10:28AM  238521        20    SSNA-2173645713   
7  Jun  1 2022 10:24AM  238520        10           NST-1038   
8  Jun  1 2022 10:24AM  238520        10           NST-1039   
9  Jun  1 2022 10:24AM  238520        10           NST-1040   

                           Customer ShipmentStatus  
0                  Eywa Pharma Inc.       Released  
1                  Eywa Pharma Inc.       Released  
2                  Eywa Pharma Inc.       Released  
3                  Eywa Pharma Inc.       Released  
4                  Eywa Pharma Inc.       Released  
5                    ClearSpec, LLC       Released  
6  Sartorius Stedim North America         Released  
7               Amcyte Pharma, Inc.       Released  
8               Amcyte Pharma, Inc.       Released  
9               Amcyte Pharma, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
60  238519       Released          3
61  238520       Released          4
62  238521       Released          1
63  238522       Released          1
64  238523       Released          5

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
238519                NaN        NaN        NaN       3.0
238520                NaN        NaN        NaN       4.0
238521                NaN        NaN        NaN       1.0
238522                NaN        NaN        NaN       1.0
238523                NaN        NaN        NaN       5.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
238318                0.0        0.0        0.0      20.0
238342                0.0        0.0        0.0       1.0
238343                0.0        0.0        0.0       1.0
238406                1.0       12.0        6.0       0.0
238409                0.0        0.0        0.0      41.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
238318                0.0          0          0        20
238342                0.0          0          0         1
238343                0.0          0          0         1
238406                1.0         12          6         0
238409                0.0          0          0        41

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Cancelled  Completed  Executing  Released
0               238318        0.0          0          0        20
1               238342        0.0          0          0         1
2               238343        0.0          0          0         1
3               238406        1.0         12          6         0
4               238409        0.0          0          0        41

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id  Cancelled Completed Executing Released
0               238318        0.0                           20
1               238342        0.0                            1
2               238343        0.0                            1
3               238406        1.0        12         6         
4               238409        0.0                           41

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                          Customer
0   Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.
5   Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC
6   Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America  
7   Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.
11  Jun  1 2022 10:20AM  238519        10                       Emerginnova
14  Jun  1 2022 10:03AM  238516        20  Sartorius Stedim North America  
15  Jun  1 2022  9:58AM  238443        15              Person & Covey, Inc.
49  Jun  1 2022  9:52AM  238515        10          Senseonics, Incorporated
50  Jun  1 2022  9:45AM  238318        15              Person & Covey, Inc.
70  Jun  1 2022  9:44AM  238512        10          Senseonics, Incorporated

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                          Customer  \
0  Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.   
1  Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC   
2  Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America     
3  Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.   
4  Jun  1 2022 10:20AM  238519        10                       Emerginnova   
5  Jun  1 2022 10:03AM  238516        20  Sartorius Stedim North America     
6  Jun  1 2022  9:58AM  238443        15              Person & Covey, Inc.   
7  Jun  1 2022  9:52AM  238515        10          Senseonics, Incorporated   
8  Jun  1 2022  9:45AM  238318        15              Person & Covey, Inc.   
9  Jun  1 2022  9:44AM  238512        10          Senseonics, Incorporated   

   Cancelled Completed Executing Released  
0        0.0                            5  
1        0.0                            1  
2        0.0                            1  
3        0.0                            4  
4        0.0                            3  
5        0.0                            1  
6        0.0                   4       30  
7        0.0                            1  
8        0.0                           20  
9        0.0                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                          Customer  \
0  Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.   
1  Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC   
2  Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America     
3  Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.   
4  Jun  1 2022 10:20AM  238519        10                       Emerginnova   
5  Jun  1 2022 10:03AM  238516        20  Sartorius Stedim North America     
6  Jun  1 2022  9:58AM  238443        15              Person & Covey, Inc.   
7  Jun  1 2022  9:52AM  238515        10          Senseonics, Incorporated   
8  Jun  1 2022  9:45AM  238318        15              Person & Covey, Inc.   
9  Jun  1 2022  9:44AM  238512        10          Senseonics, Incorporated   

  Released Executing Completed  
0        5                      
1        1                      
2        1                      
3        4                      
4        3                      
5        1                      
6       30         4            
7        1                      
8       20                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.   
1  Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC   
2  Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America     
3  Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.   
4  Jun  1 2022 10:20AM  238519        10                       Emerginnova   

  Released Executing Completed  
0        5                      
1        1                      
2        1                      
3        4                      
4        3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

Date      id Warehouse                          Customer  \
0  Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.   
1  Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC   
2  Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America     
3  Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.   
4  Jun  1 2022 10:20AM  238519        10                       Emerginnova   

   Released  Executing  Completed  
0       5.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       4.0        NaN        NaN  
4       3.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun  1 2022 10:32AM  238523        10                  Eywa Pharma Inc.   
1  Jun  1 2022 10:30AM  238522        18                    ClearSpec, LLC   
2  Jun  1 2022 10:28AM  238521        20  Sartorius Stedim North America     
3  Jun  1 2022 10:24AM  238520        10               Amcyte Pharma, Inc.   
4  Jun  1 2022 10:20AM  238519        10                       Emerginnova   

   Released  Executing  Completed  
0       5.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       4.0        0.0        0.0  
4       3.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4531173     288.0       15.0       26.0
102         953885       3.0        1.0        0.0
12          476969       2.0        1.0        1.0
15         2145972     109.0       10.0        0.0
16          238486       0.0        1.0        0.0
18          953633       3.0        1.0        0.0
19          238495      17.0       10.0        1.0
20         1430937      37.0       39.0       13.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4531173     288.0       15.0       26.0
1       102   953885       3.0        1.0        0.0
2        12   476969       2.0        1.0        1.0
3        15  2145972     109.0       10.0        0.0
4        16   238486       0.0        1.0        0.0
5        18   953633       3.0        1.0        0.0
6        19   238495      17.0       10.0        1.0
7        20  1430937      37.0       39.0       13.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     288.0       15.0       26.0
1       102       3.0        1.0        0.0
2        12       2.0        1.0        1.0
3        15     109.0       10.0        0.0
4        16       0.0        1.0        0.0
5        18       3.0        1.0        0.0
6        19      17.0       10.0        1.0
7        20      37.0       39.0       13.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  288.0
1       102  Released    3.0
2        12  Released    2.0
3        15  Released  109.0
4        16  Released    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102   12     15   16   18    19    20
Status                                                 
Completed   26.0  0.0  1.0    0.0  0.0  0.0   1.0  13.0
Executing   15.0  1.0  1.0   10.0  1.0  1.0  10.0  39.0
Released   288.0  3.0  2.0  109.0  0.0  3.0  17.0  37.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102   12     15   16   18    19    20
0          Completed   26.0  0.0  1.0    0.0  0.0  0.0   1.0  13.0
1          Executing   15.0  1.0  1.0   10.0  1.0  1.0  10.0  39.0
2           Released  288.0  3.0  2.0  109.0  0.0  3.0  17.0  37.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102   12     15   16   18    19    20
0  Completed   26.0  0.0  1.0    0.0  0.0  0.0   1.0  13.0
1  Executing   15.0  1.0  1.0   10.0  1.0  1.0  10.0  39.0
2   Released  288.0  3.0  2.0  109.0  0.0  3.0  17.0  37.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()